In [1]:
import os
import re
import inspect

# Defining functions to parse .ipf files

- `get_function_in_file` Example return {'scans.ipf':['scanfastdac', 'scanfastdac2d', ...]}
- `function_in_list` Example return ['scanfastdac', 'initscancontroller', ...]
- `get_file_names` Example return ['scans.ipf', 'scancontroller.ipf', ...]
- `function_in_functions` Example return {'scanfastdac':['sc_openinstrconnections', 'initScanVarsFD', ...]}

In [3]:
def get_function_in_file(folder_path):
    """
    Retrieves the functions defined in each file within the specified folder path.

    Args:
        folder_path (str): The path to the folder containing the files.

    Returns:
        dict: A dictionary mapping file names to the list of functions defined in each file.
    """
    dependencies = {}

    # Iterate over each file in the folder
    for filename in os.listdir(folder_path):
        functions = []
        if filename.endswith(".ipf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                function_name = ''
                for line in file:
                    line = line.strip()
                    # if line.startswith("function"):
                    if re.search(r'^(?:threadsafe\s*)?function.*', line):
                        function_name = line.split('function')[1].split('(')[0].split(' ')[-1]
                        if function_name != "":
                            functions.append(function_name.lower())

        dependencies[filename] = functions

    return dependencies


def function_in_list(function_in_file):
    """
    Returns a list of functions from a dictionary of functions in files.

    Args:
        function_in_file (dict): A dictionary mapping file names to the list of functions defined in each file.

    Returns:
        function_list: A list of functions extracted from the input dictionary.
    """
    function_list = []
    for file, functions in function_in_file.items():
        function_list += functions

    return function_list



def get_file_names(folder_path):
    """
    Retrieves the names of files with the ".ipf" extension within the specified folder.

    Args:
        folder_path (str): The path to the folder containing the files.

    Returns:
        files: A list of file names with the ".ipf" extension.

    """
    files = []

    for filename in os.listdir(folder_path):
        functions = []
        if filename.endswith(".ipf"):
            files.append(filename)

    return files



def function_in_functions(folder_path, function_name='scanfastdac'):
    """
    Retrieves the dependencies of a specific function within a folder of files.

    Args:
        folder_path (str): The path to the folder containing the files.
        function_name (str, optional): The name of the function to find dependencies for. Defaults to 'scanfastdac'.

    Returns:
        dependencies: A dictionary mapping the specified function to a list of its dependencies.

    """

    function_in_file = get_function_in_file(folder_path)
    function_list = function_in_list(function_in_file)

    dependencies = {}

    for file, functions in function_in_file.items():
        file_path = os.path.join(folder_path, file)
        for function in functions:
            if function == function_name:
                do_print = True
            else:
                do_print = False

            called_functions = []
            with open(file_path, 'r') as file:
                function_reached = False
                for line in file:
                    line = line.strip()
                    line = line.lower()
                    if re.search(r'^(?:threadsafe\s*)?function.*', line) and line.split('function')[1].split('(')[0].split(' ')[-1].lower() == function:
                        function_reached = True
                    if re.match(r'^\s*end\b *', line)  and function_reached == True:
                        function_reached = False
                    if function_reached == True:
                        # if do_print: # print every line in the function 
                        #     print(line)
                        for loop_function in function_list:
                            # if loop_function in line and loop_function != function and loop_function not in called_functions and f'[{loop_function}]' not in line and not re.search(f"\/\/.*{loop_function}", line) and not re.search(f".*s.{loop_function}", line):
                            if loop_function in line and loop_function != function and loop_function not in called_functions and f'[{loop_function}]' not in line and not re.search(f"//.*{loop_function}", line) and not re.search(f".*s.{loop_function}", line):

                                # if do_print: # print functions called within function
                                #     print(function, loop_function)
                                called_functions.append(loop_function)
            dependencies[function] = called_functions

    return dependencies

# Defining functions to create mermaid graph and save as .html
- `generate_connections` Returns list of connections between given function and all functions that are recursovely called.
- `generate_mermaid_graph` Returns a mermaid graph as a string.
- `generate_html_with_mermaid` Create .html with mermaid graph. 

In [4]:
def generate_connections(dependencies, function_name, visited_connection=None, visited_function=None):
    """
    Recursively generates the connections between given function and and all other functions that are called.

    Args:
        dependencies (dict): A dictionary mapping functions to their dependencies.
        function_name (str): The name of the function to generate connections for.
        visited_connection (list, optional): A list of visited connections to avoid duplicates. Defaults to None.
        visited_function (list, optional): A list of visited functions to avoid infinite recursion. Defaults to None.

    Returns:
        connections: A list containing a list of connections.
        dummy_functions: A list of visited functions.

    """
    if visited_connection is None:
        visited_connection = []

    if visited_function is None:
        visited_function = []
        
    dummy_functions = visited_function
    
    connections = []
    if function_name in dependencies and function_name not in visited_function:
        visited_function.append(function_name)
        called_functions = dependencies[function_name]
        for called_function in called_functions:
            connection = f"{function_name} --> {called_function};"
            if connection not in visited_connection:
                visited_connection.append(connection)
                connections.append(connection)
                generated_connections, dummy_functions = generate_connections(dependencies, called_function, visited_connection=visited_connection, visited_function=visited_function)
                connections.extend(generated_connections)
    return connections, dummy_functions



def generate_mermaid_graph(connections, visited_functions, function_in_file):
    """
    Generates a mermaid graph based on the connections between functions.

    Args:
        connections (list): A list of connections between functions.
        visited_functions (list): A list of visited functions.
        function_in_file (dict): A dictionary mapping files to their respective functions.

    Returns:
        str: The generated mermaid graph as a string.

    """
    m_graph = ['graph TB;']
    m_graph += connections
    for file, functions in function_in_file.items():
        visited = False
        m_graph.append(f'subgraph {file}')
        for function in functions:
            if function in visited_functions:
                m_graph.append(f'{function}')
                visited = True
        if visited:
            m_graph.append(f'end')
        else:
            m_graph = m_graph[:-1]
    return '\n'.join(m_graph)



def generate_html_with_mermaid(mermaid_graph, file_path=None):
    """
    Generates an HTML file with a mermaid diagram in the given file_path.

    Args:
        mermaid_graph (str): The mermaid graph as a string.
        file_path (str, optional): The file path to save the HTML file. If not provided, 'output.html' will be used.

    Returns:
        None

    """
    html_content = inspect.cleandoc(f"""
    <html>
    <body>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/zoomooz/1.4.1/zoomooz.min.js"></script>
    <style>
        .mermaid {{
            height: 600px; /* Adjust the height as needed */
            margin-bottom: 20px; /* Adjust the bottom margin as needed */
        }}
    </style>
    <pre class="mermaid">
    {mermaid_graph}
    </pre>
    <script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10.1/dist/mermaid.esm.min.mjs';
    mermaid.initialize({{
        startOnLoad: true,
        flowchart: {{
                useMaxWidth: false, // Disable maximum width for better zooming
                htmlLabels: true, // Enable HTML-based labels for better styling
                defaultRenderer: "elk", // Makes connections linear, ugly but good for large graphs
            }},
        }});
    $(document).ready(function() {{
            // Apply Zoomooz to the Mermaid diagram container
            $('.mermaid').zoomTarget();
        }});
    </script>
    </body>
    </html>
    """)


    if file_path is None:
        file_path = 'output.html'
    with open(file_path, 'w') as file:
        file.write(html_content)



# Parsing .ipf to get functions in each .ipf file

In [5]:
# Example usage
folder_path = '../Required'  # Replace with the actual folder path
function_in_file = get_function_in_file(folder_path)

# Example creating .html for one function 

In [6]:
##### Example running for one function #####
function_to_view = 'setls625fieldwait'
function_functions = function_in_functions(folder_path, function_name=function_to_view)
single_function_web, visited_functions = generate_connections(function_functions, function_name=function_to_view)
mermaid_graph = generate_mermaid_graph(single_function_web, visited_functions, function_in_file)
generate_html_with_mermaid(mermaid_graph, file_path=f'{function_to_view}.html')

# Example creating .html for EVERY FUNCTION in Repo...

In [7]:
def create_master_web(folder_path, function_in_file, ipf_path):
    if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    function_functions = function_in_functions(ipf_path)

    for file, functions_to_view in function_in_file.items():
        print(f'Starting {file}')
        full_path = f'{folder_path}/{file.split(".ipf")[0]}'
        if not os.path.exists(full_path):
            os.makedirs(full_path)            
            
        for function_to_view in functions_to_view:
            single_function_web, visited_functions = generate_connections(function_functions, function_name=function_to_view)
            mermaid_graph = generate_mermaid_graph(single_function_web, visited_functions, function_in_file)
            generate_html_with_mermaid(mermaid_graph, file_path=f'{full_path}/{function_to_view}.html')


create_master_web('html_function_webs', function_in_file, '../Required')

Starting ScanController_old.ipf
Starting structure_def.ipf
Starting hp34401A.ipf
Starting ScanController.ipf
Starting fastdac_old.ipf
Starting ScanController_SQL.ipf
Starting formatting_http.ipf
Starting ls370_resistancebridge.ipf
Starting srs830.ipf
Starting keithley2400.ipf
Starting FD_Scans.ipf
Starting babydac.ipf
Starting fastdac.ipf
Starting Scans.ipf
Starting ls625_single_supply.ipf
Starting ScanController_IO.ipf
Starting ScanController_INSTR.ipf
Starting ips120_single_supply.ipf


# Printing all the functions that call function x

In [37]:
def get_functions_that_call_functionx(function_gets_called):

    functions_that_call_x = []

    for key, val in function_functions.items():

        if function_gets_called.lower() in val:
            functions_that_call_x.append(key)

    return functions_that_call_x


def function_dependancy(func_functions):

    dependencies = {}

    for key, val in func_functions.items():

        dependencies[key] = get_functions_that_call_functionx(key)

    return dependencies


In [38]:
func_functions = function_in_functions(folder_path)
func_dependancy = function_dependancy(func_functions)

In [39]:
for key, val in func_dependancy.items():
    if (len(val) < 2):
        print(key, val)

appendvalue ['scw_addrow']
appendstring ['scw_addrow']
prompt_user ['calibratefdac']
sc_sleep_noupdate []
scv_sanitizesetpoints ['scv_setsetpoints']
scv_formatsetpoints ['scv_setsetpoints']
sci_initializewaves ['initializescan']
sci_init1dwave ['sci_initializewaves']
sci_get2dwavenames ['savetohdf']
sci_sanitycheckwavenames ['sci_initializewaves']
scg_twosubplot ['scg_initializegraphsforwavenames']
scg_graphexistsforwavename ['scg_initializegraphsforwavenames']
scg_open1dgraph ['scg_initializegraphsforwavenames']
scg_open2dgraph ['scg_initializegraphsforwavenames']
scg_getopengraphtitles ['scg_graphexistsforwavename']
scg_getopengraphids ['scg_graphexistsforwavename']
scg_openabortwindow ['initializescan']
sc_openinstrguis ['scw_openguibutton']
sc_checkbackup ['savenamedwaves']
scs_stopsweep []
scs_pausesweep ['scs_resumesweep']
scs_resumesweep ['scs_pausesweep']
sc_managethreads []
sc_worker ['sc_managethreads']
sc_funcasync ['sc_worker']
sc_killthreads []
sc_checkasyncscript ['sc_fin

In [8]:
function_gets_called = "get_dacListIDs" # CHANGE THIS STRING, I TAKE CARE OF UPPER AND LOWER CASE
functions_that_call_x = []


for function, functions_in_function in function_functions.items():
    if function_gets_called.lower() in functions_in_function:
        functions_that_call_x.append(function)


print(functions_that_call_x)

[]


In [9]:
function_functions

{'scu_assertseparatortype': [],
 'scu_unixtime': [],
 'roundnum': [],
 'appendvalue': [],
 'appendstring': [],
 'prompt_user': [],
 'ask_user': [],
 'scu_getdaclabel': ['scu_assertseparatortype', 'get_fastdac_index'],
 'scu_getchannelnumbers': ['scu_assertseparatortype',
  'removetrailingwhitespace'],
 'scu_getdevicechannels': ['scf_getfdinfofromid', 'scf_getchannelstartnum'],
 'sc_sleep': [],
 'asleep': ['sc_sleep'],
 'sc_sleep_noupdate': ['sc_sleep'],
 'scv_setlastscanvars': [],
 'scv_getlastscanvars': [],
 'initscanvars': [],
 'initscanvarsfd': ['initscanvars',
  'scf_getrecordedfadcinfo',
  'scu_getdaclabel',
  'scu_getchannelnumbers',
  'scv_setsetpoints',
  'scv_setfreq',
  'scv_setnumptssweeprateduration',
  'get_fastdac_index',
  'scv_setlastscanvars',
  'fdawg_check_awg_and_sweepgates_unique',
  'remove_fd_files'],
 'initscanvarsfd2': ['initscanvars',
  'initscanvarsfd',
  'scf_getrecordedfadcinfo',
  'scu_getdaclabel',
  'scc_getdeviceids',
  'fd_getglobalawg',
  'textwavetol